<a href="https://colab.research.google.com/github/syoone/ecolif_selenium/blob/main/ecolif_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install selenium

     |████████████████████████████████| 911kB 4.0MB/s 


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys
import random
from bs4 import BeautifulSoup
import json
import pandas as pd

In [4]:
chrome_options = Options()
#chrome_options.add_argument("--headless")

rootPath = '..'
driver = webdriver.Chrome(
    executable_path='{}/chrome/chromedriver74'.format(rootPath),
    options=chrome_options
)

#def crawl(url):
#    headers = {
#        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko)"
#    }
#    data = requests.get(url, headers)
#    print(data,url)
#    return data.content

def getProductInfo(tr):
    tds = tr.find_all('td')
    return {'no':tds[0].text, 'name':tds[1].text, 'catagory':tds[2].text, 
            'vendor':tds[3].text, 'confirmNo':tds[4].text, 'lincenceNo':tds[5].text}

def parser(pageString):
    bsObj = BeautifulSoup(pageString, 'html.parser')
    table = bsObj.find('table',{'class':'table table-striped2'})
    tbody = table.find('tbody')

    trs = tbody.find_all('tr')

    productInfos =[]
    for tr in trs:
        productInfo = getProductInfo(tr)
        productInfos.append(productInfo)

    return productInfos

url = 'http://ecolife.me.go.kr/ecolife/sntryAid/index?page=6'
driver.get(url)

result = []
def getInfos(pageNo):
    url = 'http://ecolife.me.go.kr/ecolife/sntryAid/index?page={}'.format(pageNo)
    driver.get(url)
    pageSource = driver.page_source
    infos = parser(pageSource)
    return infos

def save(df, filename):
    writer = pd.ExcelWriter(filename)
    df.to_excel(writer, 'Sheet1')
    writer.save()

for pageNo in range(1, 164 + 1):
    result = result + getInfos(pageNo)

df = pd.DataFrame(data=result)
print(df.count())
save(df, './ecolife.xlsx')

print(infos[0])

file = open('./result.json', 'w+')
file.write(json.dumps(result))

productInfos = []

def crawlPage(pageNo):
    url = 'http://ecolife.me.go.kr/ecolife/sntryAid/index?page={}'.format(pageNo)
    pageString = crawl(url)
    products = parser(pageString)
    return products


#--------------------- Main ----------------------------------------------------
result = []

for pageNo in range(1,10+1):  # pageNo = 171
    result += crawlPage(pageNo)
    df = pd.DataFrame(data=result)
df = pd.DataFrame(data=result)
save(df, './ecolife.xlsx')

print(df.count())
print(len(result))

WebDriverException: ignored